In [6]:
import re
import nltk
import emoji
import numpy as np
from nltk.tokenize import word_tokenize

In [55]:
from utils2 import get_dict




In [57]:
corpus = 'Who ❤️ "word embeddings" in 2020? I do!!!'

Cleaning data with regex.

In [62]:
cleaned_corpus = re.sub(r'[,!?;-]+', '.', corpus)
print(cleaned_corpus)

Who ❤️ "word embeddings" in 2020. I do.


In [63]:
tokenized_corpus = word_tokenize(cleaned_corpus)
print(tokenized_corpus)

['Who', '❤️', '``', 'word', 'embeddings', "''", 'in', '2020', '.', 'I', 'do', '.']


All punctuation changed to '.'



In [64]:
again_clean = [token.lower() for token in tokenized_corpus if token.isalpha() or token == '.' or emoji.get_emoji_regexp().search(token)]

In [66]:
print(again_clean)

['who', '❤️', 'word', 'embeddings', 'in', '.', 'i', 'do', '.']


In [68]:
def tokenize(corpus):
    data = re.sub(r'[,!?;-]+', '.', corpus)
    data = nltk.word_tokenize(data)  # tokenize string to words
    data = [ ch.lower() for ch in data
             if ch.isalpha()
             or ch == '.'
             or emoji.get_emoji_regexp().search(ch)
           ]
    return data

In [70]:
corpus = 'I am happy because I am learning'
print(f'Corpus:  {corpus}')
words = tokenize(corpus)
print(f'Words (tokens):  {words}')

Corpus:  I am happy because I am learning
Words (tokens):  ['i', 'am', 'happy', 'because', 'i', 'am', 'learning']


In [71]:
tokenize("Now it's your turn: try with your own sentence!")

['now', 'it', 'your', 'turn', 'try', 'with', 'your', 'own', 'sentence', '.']

# Sliding window for conitinious bag of words

C represents size of window.
Words represents input already tokenized corpus.
Output of function will yield tuple of center and context words.

In [89]:
def get_windows(words, C):
    for i in range(C, len(words) - C):
        center = words[i]
        context = words[i-C:i] + words[i+1:i+C+1]
        yield context, center 
    

In [90]:
for x, y in get_windows(
            ['i', 'am', 'happy', 'because', 'i', 'am', 'learning'],
            2
        ):
    print(f'{x}\t{y}')

['i', 'am', 'because', 'i']	happy
['am', 'happy', 'i', 'am']	because
['happy', 'because', 'am', 'learning']	i
